# R2: Delphi Comparison Using Phecode-Based ICD Mapping

This notebook creates a principled comparison between Aladynoulli and Delphi-2M by:

1. **Disease Name → Phenotype Names**: Using our `major_diseases` mapping from `evaluate_major_disease_wsex_rolling_tte.py`
2. **Phenotype Names → Phecodes**: Looking up phenotype names in the Phecode mapping file
3. **Phecodes → ICD Codes**: Extracting all ICD codes that map to those Phecodes
4. **ICD Codes → Delphi Results**: Extracting Delphi's "no gap" (t0) predictions for those ICD codes
5. **Comparison**: Our t0 predictions (`washout_0yr_results.csv`) vs Delphi's t0 predictions

This ensures we use the **actual Phecode→ICD aggregation** that our model uses, rather than manual approximations.

## Key Insight

This comparison is **more principled** than manual ICD mappings because:
- It uses the same Phecode aggregation logic that our model uses
- It captures all ICD codes that contribute to each Phecode
- It ensures fair comparison by matching on the same disease definitions

---


In [12]:
import pandas as pd
import pandas as pd
import numpy as np
from pathlib import Path
import sys


In [13]:

# Load major_diseases mapping from evaluate_major_disease_wsex_rolling_tte.py
major_diseases = {
    'ASCVD': ['Myocardial infarction', 'Coronary atherosclerosis', 'Other acute and subacute forms of ischemic heart disease', 
              'Unstable angina (intermediate coronary syndrome)', 'Angina pectoris', 'Other chronic ischemic heart disease, unspecified'],
    'Diabetes': ['Type 2 diabetes'],
    'Atrial_Fib': ['Atrial fibrillation and flutter'],
    'CKD': ['Chronic renal failure [CKD]', 'Chronic Kidney Disease, Stage III'],
    'All_Cancers': ['Colon cancer', 'Cancer of bronchus; lung', 'Cancer of prostate', 'Malignant neoplasm of bladder', 'Secondary malignant neoplasm','Secondary malignant neoplasm of digestive systems', 'Secondary malignant neoplasm of liver'],
    'Stroke': ['Cerebral artery occlusion, with cerebral infarction', 'Cerebral ischemia'],
    'Heart_Failure': ['Congestive heart failure (CHF) NOS', 'Heart failure NOS'],
    'Pneumonia': ['Pneumonia', 'Bacterial pneumonia', 'Pneumococcal pneumonia'],
    'COPD': ['Chronic airway obstruction', 'Emphysema', 'Obstructive chronic bronchitis'],
    'Osteoporosis': ['Osteoporosis NOS'],
    'Anemia': ['Iron deficiency anemias, unspecified or not due to blood loss', 'Other anemias'],
    'Colorectal_Cancer': ['Colon cancer', 'Malignant neoplasm of rectum, rectosigmoid junction, and anus'],
    'Breast_Cancer': ['Breast cancer [female]', 'Malignant neoplasm of female breast'],
    'Prostate_Cancer': ['Cancer of prostate'],
    'Lung_Cancer': ['Cancer of bronchus; lung'],
    'Bladder_Cancer': ['Malignant neoplasm of bladder'],
    'Secondary_Cancer': ['Secondary malignant neoplasm', 'Secondary malignancy of lymph nodes', 'Secondary malignancy of respiratory organs', 'Secondary malignant neoplasm of digestive systems'],
    'Depression': ['Major depressive disorder'],
    'Anxiety': ['Anxiety disorder'],
    'Bipolar_Disorder': ['Bipolar'],
    'Rheumatoid_Arthritis': ['Rheumatoid arthritis'],
    'Psoriasis': ['Psoriasis vulgaris'],
    'Ulcerative_Colitis': ['Ulcerative colitis'],
    'Crohns_Disease': ['Regional enteritis'],
    'Asthma': ['Asthma'],
    'Parkinsons': ["Parkinson's disease"],
    'Multiple_Sclerosis': ['Multiple sclerosis'],
    'Thyroid_Disorders': ['Thyrotoxicosis with or without goiter', 'Secondary hypothyroidism', 'Hypothyroidism NOS']
}

print(f"✓ Loaded {len(major_diseases)} disease mappings")
print(f"\nExample mappings:")
for disease, phenotypes in list(major_diseases.items())[:3]:
    print(f"  {disease}: {phenotypes}")

✓ Loaded 28 disease mappings

Example mappings:
  ASCVD: ['Myocardial infarction', 'Coronary atherosclerosis', 'Other acute and subacute forms of ischemic heart disease', 'Unstable angina (intermediate coronary syndrome)', 'Angina pectoris', 'Other chronic ischemic heart disease, unspecified']
  Diabetes: ['Type 2 diabetes']
  Atrial_Fib: ['Atrial fibrillation and flutter']



## Step 2: Load Phecode Mapping File

Load the Phecode mapping file that contains ICD-10 → Phecode mappings. This file also contains phenotype names that we can match against.

In [14]:
# Try to load Phecode mapping file
phecode_mapping_paths = [
    '/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/icd2phecode_mergedwithdetailedphecode.rds',
    '/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/icd2phecode_mergedwithdetailedphecode.csv',
    '/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/icd2phecode_mergedwithdetailedphecode_info.csv',
]

phecode_mapping_df = None
for path in phecode_mapping_paths:
    if Path(path).exists():
        if path.endswith('.rds'):
            if HAS_PYREADR:
                try:
                    result = pyreadr.read_r(str(path))
                    phecode_mapping_df = result[None]
                    print(f"✓ Loaded Phecode mapping from RDS: {len(phecode_mapping_df)} rows")
                    break
                except Exception as e:
                    print(f"⚠️  Error loading RDS: {e}")
            else:
                print("⚠️  pyreadr not available, trying CSV...")
        else:
            try:
                phecode_mapping_df = pd.read_csv(path)
                print(f"✓ Loaded Phecode mapping from CSV: {len(phecode_mapping_df)} rows")
                break
            except Exception as e:
                print(f"⚠️  Error loading CSV: {e}")

if phecode_mapping_df is None:
    print("⚠️  Phecode mapping file not found!")
    print("   Expected locations:")
    for path in phecode_mapping_paths:
        print(f"     {path}")
else:
    print(f"\nColumns in mapping file: {phecode_mapping_df.columns.tolist()}")
    print(f"\nFirst few rows:")
    print(phecode_mapping_df.head())


✓ Loaded Phecode mapping from RDS: 4011590 rows

Columns in mapping file: ['diag_icd10', 'eid', 'age_diag', 'phecode', 'phenotype', 'exclude_range', 'exclude_name']

First few rows:
  diag_icd10      eid   age_diag  phecode             phenotype exclude_range  \
0       A009  2029620  64.294512      8.0  Intestinal infection    001-009.99   
1       A009  1308023  42.463391      8.0  Intestinal infection    001-009.99   
2       A009  2019059  65.331874      8.0  Intestinal infection    001-009.99   
3       A009  2019059  65.329136      8.0  Intestinal infection    001-009.99   
4       A009  2019059  65.331874      8.0  Intestinal infection    001-009.99   

          exclude_name  
0  infectious diseases  
1  infectious diseases  
2  infectious diseases  
3  infectious diseases  
4  infectious diseases  



## Step 3: Map Phenotype Names → Phecodes → ICD Codes

For each disease, find the Phecodes that match its phenotype names, then extract all ICD codes that map to those Phecodes.

In [15]:
# Identify column names in mapping file
if phecode_mapping_df is not None:
    # Common column name variations
    icd_col = None
    phecode_col = None
    phenotype_col = None
    
    for col in phecode_mapping_df.columns:
        col_lower = col.lower()
        if 'icd' in col_lower or 'diag' in col_lower:
            icd_col = col
        if 'phecode' in col_lower or 'phe_code' in col_lower:
            phecode_col = col
        if 'phenotype' in col_lower or 'description' in col_lower or 'name' in col_lower:
            phenotype_col = col
    
    print(f"Identified columns:")
    print(f"  ICD column: {icd_col}")
    print(f"  Phecode column: {phecode_col}")
    print(f"  Phenotype column: {phenotype_col}")
    
    if icd_col is None or phecode_col is None:
        print("\n⚠️  Could not identify required columns. Available columns:")
        print(f"   {phecode_mapping_df.columns.tolist()}")
    else:
        print(f"\n✓ Found required columns for mapping")
else:
    print("⚠️  Cannot proceed without Phecode mapping file")

Identified columns:
  ICD column: age_diag
  Phecode column: phecode
  Phenotype column: exclude_name

✓ Found required columns for mapping


In [19]:
phecode_mapping_df

,diag_icd10,eid,age_diag,phecode,phenotype,exclude_range,exclude_name
0,A009,2029620,64.294512,8.0,Intestinal infection,001-009.99,infectious diseases
1,A009,1308023,42.463391,8.0,Intestinal infection,001-009.99,infectious diseases
2,A009,2019059,65.331874,8.0,Intestinal infection,001-009.99,infectious diseases
3,A009,2019059,65.329136,8.0,Intestinal infection,001-009.99,infectious diseases
4,A009,2019059,65.331874,8.0,Intestinal infection,001-009.99,infectious diseases
...,...,...,...,...,...,...,...
4011585,Z991,2357478,70.937457,509.8,Dependence on respirator [Ventilator] or suppl...,500-509.99,respiratory
4011586,Z991,1411604,72.637197,509.8,Dependence on respirator [Ventilator] or suppl...,500-509.99,respiratory
4011587,Z991,1570445,57.331326,509.8,Dependence on respirator [Ventilator] or suppl...,500-509.99,respiratory
4011588,Z991,2093991,57.136992,509.8,Dependence on respirator [Ventilator] or suppl...,500-509.99,respiratory


In [20]:
# Map disease → phenotype → ICD10 codes directly from phecode_mapping_df
# For each disease, find rows where 'phenotype' column matches phenotype names, then extract 'diag_icd10' codes
disease_to_icd_mapping = {}

if phecode_mapping_df is not None:
    # Use correct column names from the mapping file
    phenotype_col = 'phenotype'  # Column with phenotype names
    icd10_col = 'diag_icd10'  # Column with ICD10 codes
    
    for disease_name, phenotype_list in major_diseases.items():
        matched_icd10_codes = set()
        matched_phenotypes = []
        
        # For each phenotype in the disease, find matching rows in phecode_mapping_df
        for phenotype in phenotype_list:
            # Match phenotype name in the 'phenotype' column (case-insensitive)
            matches = phecode_mapping_df[
                phecode_mapping_df[phenotype_col].str.contains(phenotype, case=False, na=False, regex=False)
            ]
            
            if len(matches) > 0:
                # Extract unique ICD10 codes from matching rows
                icd10_codes = matches[icd10_col].dropna().unique()
                matched_icd10_codes.update(icd10_codes)
                matched_phenotypes.append(phenotype)
                print(f"  ✓ {phenotype}: {len(icd10_codes)} ICD10 codes")
        
        disease_to_icd_mapping[disease_name] = {
            'phenotypes': phenotype_list,
            'matched_phenotypes': matched_phenotypes,
            'icd10_codes': sorted(list(matched_icd10_codes))
        }
        
        print(f"{disease_name}: {len(matched_phenotypes)}/{len(phenotype_list)} phenotypes matched, {len(matched_icd10_codes)} unique ICD10 codes")
    
    print(f"\n✓ Mapped {len(disease_to_icd_mapping)} diseases to ICD10 codes")
    
    # Show summary
    print("\nSummary:")
    for disease, mapping in list(disease_to_icd_mapping.items())[:5]:
        print(f"  {disease}: {len(mapping['icd10_codes'])} ICD10 codes")
        if len(mapping['icd10_codes']) > 0:
            print(f"    Examples: {mapping['icd10_codes'][:5]}")
else:
    print("⚠️  Cannot create mapping without Phecode file")



  ✓ Myocardial infarction: 1 ICD10 codes
  ✓ Other acute and subacute forms of ischemic heart disease: 1 ICD10 codes
  ✓ Other chronic ischemic heart disease, unspecified: 1 ICD10 codes
ASCVD: 3/6 phenotypes matched, 3 unique ICD10 codes
  ✓ Type 2 diabetes: 2 ICD10 codes
Diabetes: 1/1 phenotypes matched, 2 unique ICD10 codes
  ✓ Atrial fibrillation and flutter: 1 ICD10 codes
Atrial_Fib: 1/1 phenotypes matched, 1 unique ICD10 codes
  ✓ Chronic renal failure [CKD]: 1 ICD10 codes
  ✓ Chronic Kidney Disease, Stage III: 1 ICD10 codes
CKD: 2/2 phenotypes matched, 2 unique ICD10 codes
  ✓ Colon cancer: 1 ICD10 codes
  ✓ Cancer of prostate: 1 ICD10 codes
  ✓ Malignant neoplasm of bladder: 1 ICD10 codes
  ✓ Secondary malignant neoplasm: 2 ICD10 codes
  ✓ Secondary malignant neoplasm of liver: 1 ICD10 codes
All_Cancers: 5/7 phenotypes matched, 5 unique ICD10 codes
  ✓ Cerebral ischemia: 1 ICD10 codes
Stroke: 1/2 phenotypes matched, 1 unique ICD10 codes
  ✓ Congestive heart failure (CHF) NOS: 1 

In [21]:
# Load Delphi supplementary table
delphi_supp_paths = [
    '/Users/sarahurbut/Downloads/41586_2025_9529_MOESM3_ESM.csv',
    '/Users/sarahurbut/aladynoulli2/claudefile/output/delphi_supplementary.csv',
]

delphi_supp = None
for path in delphi_supp_paths:
    if Path(path).exists():
        try:
            delphi_supp = pd.read_csv(path)
            print(f"✓ Loaded Delphi supplementary table: {len(delphi_supp)} rows")
            print(f"  Columns: {delphi_supp.columns.tolist()}")
            break
        except Exception as e:
            print(f"⚠️  Error loading {path}: {e}")

if delphi_supp is None:
    print("⚠️  Delphi supplementary table not found!")
    print("   Expected locations:")
    for path in delphi_supp_paths:
        print(f"     {path}")
else:
    print(f"\nFirst few rows:")
    print(delphi_supp.head())


✓ Loaded Delphi supplementary table: 1270 rows
  Columns: ['Index', 'Name', 'ICD-10 Chapter', 'ICD-10 Chapter (short)', 'Colour', 'AUC Female, (no gap)', 'AUC Male, (no gap)', 'AUC Female, (1 year gap)', 'AUC Male, (1 year gap)', 'N tokens, training', 'N tokens, validation']

First few rows:
   Index     Name            ICD-10 Chapter    ICD-10 Chapter (short)  \
0      0  Padding                 Technical                 Technical   
1      1  Healthy                 Technical                 Technical   
2      2   Female                       Sex                       Sex   
3      3     Male                       Sex                       Sex   
4      4  BMI low  Smoking, Alcohol and BMI  Smoking, Alcohol and BMI   

    Colour  AUC Female, (no gap)  AUC Male, (no gap)  \
0  #2a52be                   NaN                 NaN   
1  #2a52be                   NaN                 NaN   
2  #bcbd22                   NaN                 NaN   
3  #bcbd22                   NaN            

## Step 5: Extract Delphi Results Using ICD10 Codes

Match the ICD10 codes from our phecode mapping against Delphi's table to extract AUC results.


In [22]:
# Extract Delphi AUCs for each disease using ICD10 codes from phecode mapping
# Match ICD10 codes from disease_to_icd_mapping against Delphi table
delphi_results = []

if delphi_supp is not None and 'disease_to_icd_mapping' in locals():
    # Identify Delphi column names
    name_col = None
    auc_0gap_female_col = None
    auc_0gap_male_col = None
    
    for col in delphi_supp.columns:
        col_lower = col.lower()
        if 'name' in col_lower and name_col is None:
            name_col = col
        if 'auc' in col_lower and 'female' in col_lower and 'no gap' in col_lower:
            auc_0gap_female_col = col
        if 'auc' in col_lower and 'male' in col_lower and 'no gap' in col_lower:
            auc_0gap_male_col = col
    
    print(f"Delphi columns identified:")
    print(f"  Name: {name_col}")
    print(f"  AUC Female (0 gap): {auc_0gap_female_col}")
    print(f"  AUC Male (0 gap): {auc_0gap_male_col}")
    
    if name_col:
        for disease_name, mapping_info in disease_to_icd_mapping.items():
            icd10_codes = mapping_info['icd10_codes']
            
            if len(icd10_codes) == 0:
                print(f"⚠️  {disease_name}: No ICD10 codes found from phecode mapping")
                continue
            
            # Match ICD10 codes against Delphi table
            # Delphi Name column contains ICD codes like "I21 Acute myocardial infarction"
            matching_rows = []
            matched_icd10_codes = []
            
            for icd10_code in icd10_codes:
                # Match ICD codes that start with the pattern (e.g., "I21" matches "I21 Acute myocardial infarction")
                matches = delphi_supp[
                    delphi_supp[name_col].str.contains(f'^{icd10_code}', regex=True, case=False, na=False)
                ]
                
                if len(matches) > 0:
                    matching_rows.append(matches)
                    matched_icd10_codes.append(icd10_code)
                    print(f"  ✓ {disease_name}: ICD10 {icd10_code} → {len(matches)} Delphi matches")
            
            if len(matching_rows) > 0:
                # Combine all matching rows
                combined = pd.concat(matching_rows).drop_duplicates()
                
                # Create one row per Delphi ICD code match (1-to-many structure)
                import re
                
                for idx, row in combined.iterrows():
                    # Extract ICD code and name from Delphi row
                    icd_code = None
                    delphi_name = None
                    if name_col in row.index:
                        name_val = str(row[name_col])
                        delphi_name = name_val
                        icd_match = re.match(r'^([A-Z]\d{2})', name_val)
                        if icd_match:
                            icd_code = icd_match.group(1)
                    
                    # Collect both female and male AUCs
                    female_auc = None
                    male_auc = None
                    
                    if auc_0gap_female_col and auc_0gap_female_col in row.index and pd.notna(row[auc_0gap_female_col]):
                        female_auc = row[auc_0gap_female_col]
                    
                    if auc_0gap_male_col and auc_0gap_male_col in row.index and pd.notna(row[auc_0gap_male_col]):
                        male_auc = row[auc_0gap_male_col]
                    
                    # Average male and female if both available, otherwise use available one
                    if female_auc is not None and male_auc is not None:
                        avg_auc = (female_auc + male_auc) / 2
                    elif female_auc is not None:
                        avg_auc = female_auc
                    elif male_auc is not None:
                        avg_auc = male_auc
                    else:
                        continue  # Skip if no AUC available
                    
                    # Create one row per Delphi ICD code match
                    delphi_results.append({
                        'Disease': disease_name,
                        'Delphi_t0': avg_auc,
                        'Delphi_ICD_code': icd_code if icd_code else '',
                        'Delphi_name': delphi_name if delphi_name else '',
                        'Delphi_female_auc': female_auc if female_auc is not None else np.nan,
                        'Delphi_male_auc': male_auc if male_auc is not None else np.nan,
                        'N_ICD10_codes_matched': len(matched_icd10_codes),
                        'N_ICD10_codes_total': len(icd10_codes),
                        'Matched_ICD10_codes': ', '.join(matched_icd10_codes[:5]) + ('...' if len(matched_icd10_codes) > 5 else '')
                    })
            else:
                print(f"⚠️  {disease_name}: No Delphi matches found for {len(icd10_codes)} ICD10 codes")
    
    delphi_df = pd.DataFrame(delphi_results)
    print(f"\n" + "="*80)
    print(f"✓ Extracted Delphi results: {len(delphi_df)} ICD code matches across {len(set(delphi_df['Disease']))} diseases")
    print("="*80)
    if len(delphi_df) > 0:
        print(f"\nDelphi results summary (1-to-many structure):")
        print(f"  Total Delphi ICD code matches: {len(delphi_df)}")
        print(f"  Unique diseases: {len(set(delphi_df['Disease']))}")
        print(f"\nExample (showing all ICD codes for first disease):")
        first_disease = delphi_df['Disease'].iloc[0]
        print(delphi_df[delphi_df['Disease'] == first_disease][['Disease', 'Delphi_ICD_code', 'Delphi_t0', 'Delphi_name']].to_string(index=False))
else:
    print("⚠️  Cannot extract Delphi results without Delphi table or disease_to_icd_mapping")


Delphi columns identified:
  Name: Name
  AUC Female (0 gap): AUC Female, (no gap)
  AUC Male (0 gap): AUC Male, (no gap)
⚠️  ASCVD: No Delphi matches found for 3 ICD10 codes
⚠️  Diabetes: No Delphi matches found for 2 ICD10 codes
  ✓ Atrial_Fib: ICD10 I48 → 1 Delphi matches
⚠️  CKD: No Delphi matches found for 2 ICD10 codes
⚠️  All_Cancers: No Delphi matches found for 5 ICD10 codes
⚠️  Stroke: No Delphi matches found for 1 ICD10 codes
⚠️  Heart_Failure: No Delphi matches found for 1 ICD10 codes
  ✓ Pneumonia: ICD10 J13 → 1 Delphi matches
⚠️  COPD: No Delphi matches found for 5 ICD10 codes
⚠️  Osteoporosis: No Delphi matches found for 1 ICD10 codes
⚠️  Anemia: No Delphi matches found for 1 ICD10 codes
⚠️  Colorectal_Cancer: No Delphi matches found for 1 ICD10 codes
⚠️  Breast_Cancer: No ICD10 codes found from phecode mapping
⚠️  Prostate_Cancer: No Delphi matches found for 1 ICD10 codes
⚠️  Lung_Cancer: No ICD10 codes found from phecode mapping
⚠️  Bladder_Cancer: No Delphi matches fou

## Step 6: Load Aladynoulli t0 Predictions

Load our t0 predictions from `washout_0yr_results.csv` (predictions at enrollment, 0-year washout).


In [ ]:
# Load Aladynoulli t0 predictions (0-year washout = predictions at enrollment)
aladynoulli_t0_path = Path('/Users/sarahurbut/aladynoulli2/pyScripts/dec_6_revision/new_notebooks/results/washout/pooled_retrospective/washout_0yr_results.csv')

if aladynoulli_t0_path.exists():
    aladynoulli_t0 = pd.read_csv(aladynoulli_t0_path)
    aladynoulli_t0 = aladynoulli_t0[['Disease', 'AUC']].copy()
    aladynoulli_t0.columns = ['Disease', 'Aladynoulli_t0']
    
    print(f"✓ Loaded Aladynoulli t0 predictions for {len(aladynoulli_t0)} diseases")
    print(f"\nTop 10 diseases by AUC:")
    print(aladynoulli_t0.nlargest(10, 'Aladynoulli_t0')[['Disease', 'Aladynoulli_t0']].to_string(index=False))
else:
    print(f"⚠️  Aladynoulli results file not found: {aladynoulli_t0_path}")
    aladynoulli_t0 = None


## Step 7: Compare Aladynoulli vs Delphi (t0 predictions)

Compare our t0 predictions (0-year washout) against Delphi's t0 predictions (no gap) using the 1-to-many structure.


In [ ]:
# Merge and compare (1-to-many structure)
# Our t0 predictions vs Delphi's t0 predictions (no gap)
if aladynoulli_t0 is not None and len(delphi_results) > 0:
    # Merge: our 1 prediction per disease with ALL Delphi ICD code matches
    comparison = aladynoulli_t0.merge(
        delphi_df[['Disease', 'Delphi_t0', 'Delphi_ICD_code', 'Delphi_name']],
        on='Disease',
        how='inner'
    )
    
    comparison['Advantage'] = comparison['Aladynoulli_t0'] - comparison['Delphi_t0']
    comparison = comparison.sort_values(['Disease', 'Advantage'], ascending=[True, False])
    
    print("="*80)
    print("ALADYNOULLI vs DELPHI: t0 PREDICTIONS (1-to-Many Comparison)")
    print("="*80)
    print(f"\n{len(comparison)} comparisons ({len(set(comparison['Disease']))} diseases)")
    print(f"  Our model: 1 aggregated prediction per disease (0-year washout)")
    print(f"  Delphi: Multiple ICD code predictions per disease (no gap)")
    
    # Count wins: our prediction beats at least one Delphi ICD code
    wins_by_disease = comparison.groupby('Disease')['Advantage'].apply(lambda x: (x > 0).any())
    n_wins = wins_by_disease.sum()
    n_diseases = len(wins_by_disease)
    
    print(f"\nAladynoulli wins (beats at least one Delphi ICD code): {n_wins}/{n_diseases} diseases ({n_wins/n_diseases*100:.1f}%)")
    
    # Count how many Delphi ICD codes we beat per disease
    beats_count = comparison.groupby('Disease')['Advantage'].apply(lambda x: (x > 0).sum())
    total_delphi_codes = comparison.groupby('Disease').size()
    
    print(f"\nMean advantage: {comparison['Advantage'].mean():.4f}")
    print(f"Median advantage: {comparison['Advantage'].median():.4f}")
    
    print("\n" + "-"*80)
    print("Example: ASCVD (showing all Delphi ICD code comparisons):")
    print("-"*80)
    if 'ASCVD' in comparison['Disease'].values:
        ascdv_comparison = comparison[comparison['Disease'] == 'ASCVD']
        print(ascdv_comparison[['Disease', 'Aladynoulli_t0', 'Delphi_ICD_code', 'Delphi_t0', 'Advantage']].to_string(index=False))
    
    # Save results (1-to-many structure)
    output_dir = Path('/Users/sarahurbut/aladynoulli2/pyScripts/dec_6_revision/new_notebooks/results/comparisons/pooled_retrospective')
    output_dir.mkdir(parents=True, exist_ok=True)
    
    comparison_save = comparison.copy()
    comparison_save['Win?'] = comparison_save['Advantage'].apply(lambda x: '✓' if x > 0 else '✗')
    comparison_save = comparison_save.sort_values(['Disease', 'Advantage'], ascending=[True, False])
    comparison_save.to_csv(output_dir / 'delphi_comparison_phecode_mapping_t0_1tomany.csv', index=False)
    print(f"\n✓ Results saved to: {output_dir / 'delphi_comparison_phecode_mapping_t0_1tomany.csv'}")
else:
    print("⚠️  Cannot create comparison without both Aladynoulli and Delphi results")
